# Weighted Average

https://docs.xarray.dev/en/stable/examples/area_weighted_temperature.html

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

In [ ]:
from pathlib import Path
data_dir = Path("/Users/pingu/tmp")

In [ ]:
cmip6_files = sorted(data_dir.glob('**/*.nc'))
cmip6_files

In [ ]:
ds = xr.open_dataset(cmip6_files[0])
ds

In [ ]:
weights = np.cos(np.deg2rad(ds.lat))
weights.name = "weights"
weights

In [ ]:
ds_weighted = ds.weighted(weights)
ds_weighted

In [ ]:
weighted_mean = ds_weighted.mean(("lon", "lat"))
weighted_mean